# Merging Results - qmd and shiny csv

In [38]:
import pandas as pd
import os

metrics_dir = os.path.join("..", "metrics")

rf_all = pd.read_csv(os.path.join(metrics_dir, "rf_augmented_metrics.csv"))

rf_pixel = rf_all[(rf_all["HOG"] == False) & (rf_all["PCA"] == False)].copy()
rf_pixel["Model_Label"] = "RF (pixels)"

rf_pca = rf_all[(rf_all["PCA"] == True)].copy()
rf_pca["Model_Label"] = "RF (PCA)"

rf_hog = rf_all[(rf_all["HOG"] == True)].copy()
rf_hog["Model_Label"] = "RF (HOG)"

# dropping flag column
for df in [rf_pixel, rf_pca, rf_hog]:
    df.drop(columns=["HOG", "PCA"], inplace=True, errors="ignore")

xgb_df = pd.read_csv(os.path.join(metrics_dir, "xgboost_augmented_metrics.csv"))
xgb_df = xgb_df[xgb_df["noise_level"] != 50]
xgb_df["Model_Label"] = "XGBoost (PCA)"

cnn_df = pd.read_csv(os.path.join(metrics_dir, "cnn_original_augmented_metrics.csv"))
cnn_df["Model_Label"] = "CNN"

resnet_df = pd.read_csv(os.path.join(metrics_dir, "resnet50_augmented_metrics.csv"))
resnet_df["Model_Label"] = "ResNet"

# fixing column names, making sure a confusion matrix always exists
for df in [xgb_df, cnn_df, resnet_df]:
    df.rename(columns={
        "precision_overall": "precision",
        "recall_overall": "recall",
        "confusion": "confusion_matrix"
    }, inplace=True, errors='ignore')
    if "confusion_matrix" not in df.columns:
        df["confusion_matrix"] = "[]"

# combining all for report and csv
combined_all = pd.concat([rf_pixel, rf_pca, rf_hog, xgb_df, cnn_df, resnet_df], ignore_index=True)
combined_all.fillna(0.0, inplace=True)
combined_all.drop(columns=["model_label"], inplace=True, errors="ignore")

# combined_all.head()
combined_all.to_csv(os.path.join(metrics_dir, "combined_report_metrics.csv"), index=False)

# app csv only (test set 1, PCA only)
rf_pca_shiny = rf_pca[rf_pca["test_set"] == 1].copy()

xgb_shiny = xgb_df[xgb_df["test_set"] == 1].copy()
cnn_shiny = cnn_df[cnn_df["test_set"] == 1].copy()
resnet_shiny = resnet_df[resnet_df["test_set"] == 1].copy()

combined_shiny = pd.concat([rf_pca_shiny, xgb_shiny, cnn_shiny, resnet_shiny], ignore_index=True)
combined_shiny.fillna(0.0, inplace=True)
combined_shiny.drop(columns=["test_set"], inplace=True, errors="ignore")
combined_shiny.drop(columns=["model_label"], inplace=True, errors="ignore")

# combined_shiny.head()
combined_shiny.to_csv(os.path.join(metrics_dir, "combined_app_metrics.csv"), index=False)